# 大数据处理技巧

In [ ]:
- float64转float32
- object转category
- date 'YYYY-MM-HH'转uint32

In [5]:
import pandas as pd

In [12]:
gl = pd.read_csv('../datasets/titanic.csv')
gl.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# 数据的条数与特征值 891条数据 * 12个特征值
gl.shape

(891, 12)

In [24]:
# 查看内存大小 memory usage: 285.6 KB 
# dtypes 列的数据类型
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 285.6 KB


In [28]:
for dtype in ['float64','int64','object']:
        selected_dtype = gl.select_dtypes(include=[dtype])
        mean_usage_b=selected_dtype.memory_usage(deep=True).mean()
        # 对应列平均内存占用情况
        mean_usage_mb=mean_usage_b/1024**2
        print('平均内存占用',dtype,mean_usage_mb)

平均内存占用 float64 0.004573822021484375
平均内存占用 int64 0.0056858062744140625
平均内存占用 object 0.038555145263671875


In [30]:
# 不同数的类型的数字范围
import numpy as np
int_types = ['uint8','int8','int16','int32','int64']
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



In [40]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb=usage_b/1024**2
    return '{:03.2f} MB'.format(usage_mb)
# gl.info()
gl_int =gl.select_dtypes(include=['int64'])
coverted_int=gl_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(gl_int))
print(mem_usage(coverted_int))

0.03 MB
0.01 MB


In [54]:
gl_float =gl.select_dtypes(include=['float64'])
coverted_float=gl_int.apply(pd.to_numeric,downcast='float')
print(mem_usage(gl_float))
print(mem_usage(coverted_float))

0.01 MB
0.01 MB


In [60]:
optimized_gl=gl.copy()
optimized_gl[coverted_int.columns]=coverted_int
optimized_gl[coverted_float.columns]=coverted_float
print(mem_usage(gl))
print(mem_usage(optimized_gl))



0.28 MB
0.27 MB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250000,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283302,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925000,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.099998,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.000000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.000000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.450001,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.000000,C148,C


In [62]:
optimized_gl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float32
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float32
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float32(2), int64(5), object(5)
memory usage: 76.7+ KB


In [66]:
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


In [76]:
sex=gl_obj.Sex
sex.head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [84]:
# 将object 转 category
sex_cat = sex.astype('category')
sex_cat.head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: category
Categories (2, object): ['female', 'male']

In [88]:
# 查看category的编码格式
sex_cat.head(10).cat.codes

0    1
1    0
2    0
3    0
4    1
5    1
6    1
7    1
8    0
9    0
dtype: int8

In [90]:
# 转化前 
print(mem_usage(sex))
# 转化后
print(mem_usage(sex_cat))

0.05 MB
0.00 MB


In [130]:
converted_obj = pd.DataFrame()

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5 :
        converted_obj.loc[:,col]=gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col]=gl_obj[col]
gl_obj.info()

#76KB->34.9KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      891 non-null    object
 1   Sex       891 non-null    object
 2   Ticket    891 non-null    object
 3   Cabin     204 non-null    object
 4   Embarked  889 non-null    object
dtypes: object(5)
memory usage: 34.9+ KB


In [110]:
# converted_obj.loc[:,'Sex']
# converted_obj['Sex'].astype('category')
# gl_obj.columns

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: category
Categories (2, object): ['female', 'male']

In [7]:
# date = optimized_gl.date
# optimized_gl['date'] = pd.to_datetime(date,format='%Y%m%d')
# 18710504 int32 内存占用小
# 1871-05-04 标准时间格式 内存占用大

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa8 in position 8108: invalid start byte